In [1]:
import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
import pickle


yolo = YOLO("yolov8n.pt")
COCO_CLASSES = yolo.names

mobilenet_model = tf.keras.models.load_model(r"C:\Users\moham\Downloads\Robotech\Summer_Training\Project\Note_Book\Big_data_Model\Big_Model_Final.h5")

with open(r"C:\Users\moham\Downloads\Robotech\Summer_Training\Project\Note_Book\Big_data_Model\label_dict1.pkl", "rb") as f:
    label_dict = pickle.load(f)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo(frame, stream=True)

    for r in results:
        for box in r.boxes:
            
            class_id = int(box.cls[0])
            label = COCO_CLASSES[class_id]

            if label != "person":
                continue
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

            hand_img = frame[y1:y2, x1:x2]

            if hand_img.size > 0:

                hand_resized = cv2.resize(hand_img, (224, 224))
                hand_resized = hand_resized.astype("float32") / 255.0
                hand_resized = np.expand_dims(hand_resized, axis=0)

                preds = mobilenet_model.predict(hand_resized)
                class_id = int(np.argmax(preds))
                confidence = float(np.max(preds))

                if confidence > 0.6:
                    letter = label_dict[class_id]
                    label = f"{letter}: {confidence:.2f}"

                    cv2.putText(frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)


    cv2.imshow("Sign Language Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\MoDo\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'C:\Users\moham\Downloads\Robotech\Summer_Training\Project\Note_Book\Big_data_Model\Big_Model_Final.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)